[View in Colaboratory](https://colab.research.google.com/github/upadhyayprakash/EIP/blob/master/4B_4_Try_79_48_137794.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# Added by Prakash
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    flat = Flatten()(relu)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#Last_Block = add_denseblock(Third_Block,  num_filter, dropout_rate)

output = output_layer(Third_Block)

In [30]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 32, 32, 12)   324         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 32, 32, 12)   48          conv2d_79[0][0]                  
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 32, 32, 12)   0           batch_normalization_79[0][0]     
__________________________________________________________________________________________________
conv2d_80 

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [32]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 160s 3ms/step - loss: 1.6885 - acc: 0.3963 - val_loss: 1.4273 - val_acc: 0.4956
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:27 - loss: 1.3571 - acc: 0.5146

50000/50000 [==============================] - 149s 3ms/step - loss: 1.2936 - acc: 0.5380 - val_loss: 1.7514 - val_acc: 0.4277
Epoch 3/50
45568/50000 [==========================>...] - ETA: 12s - loss: 1.1421 - acc: 0.5951

50000/50000 [==============================] - 149s 3ms/step - loss: 1.1400 - acc: 0.5955 - val_loss: 1.1679 - val_acc: 0.6022
Epoch 4/50
50000/50000 [==============================] - 149s 3ms/step - loss: 1.0328 - acc: 0.6363 - val_loss: 1.0472 - val_acc: 0.6484
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:06 - loss: 0.9569 - acc: 0.6629

50000/50000 [==============================] - 147s 3ms/step - loss: 0.9464 - acc: 0.6666 - val_loss: 1.0524 - val_acc: 0.6472
Epoch 6/50
39552/50000 [======================>.......] - ETA: 29s - loss: 0.8792 - acc: 0.6916

50000/50000 [==============================] - 150s 3ms/step - loss: 0.8787 - acc: 0.6910 - val_loss: 0.9723 - val_acc: 0.6741
Epoch 7/50
50000/50000 [==============================] - 150s 3ms/step - loss: 0.8312 - acc: 0.7092 - val_loss: 1.1534 - val_acc: 0.6541
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:17 - loss: 0.8044 - acc: 0.7245

...

50000/50000 [==============================] - 149s 3ms/step - loss: 0.3666 - acc: 0.8680 - val_loss: 0.7851 - val_acc: 0.7728
Epoch 45/50
38656/50000 [======================>.......] - ETA: 31s - loss: 0.3572 - acc: 0.8730

50000/50000 [==============================] - 148s 3ms/step - loss: 0.3602 - acc: 0.8717 - val_loss: 0.7006 - val_acc: 0.7957
Epoch 46/50
50000/50000 [==============================] - 148s 3ms/step - loss: 0.3625 - acc: 0.8696 - val_loss: 0.6594 - val_acc: 0.8013
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:17 - loss: 0.3265 - acc: 0.8841

50000/50000 [==============================] - 149s 3ms/step - loss: 0.3617 - acc: 0.8702 - val_loss: 0.7207 - val_acc: 0.7920
Epoch 48/50
38656/50000 [======================>.......] - ETA: 32s - loss: 0.3524 - acc: 0.8730

50000/50000 [==============================] - 149s 3ms/step - loss: 0.3548 - acc: 0.8713 - val_loss: 0.6879 - val_acc: 0.7947
Epoch 49/50
50000/50000 [==============================] - 149s 3ms/step - loss: 0.3535 - acc: 0.8737 - val_loss: 0.6907 - val_acc: 0.7956
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:16 - loss: 0.3620 - acc: 0.8730

50000/50000 [==============================] - 147s 3ms/step - loss: 0.3455 - acc: 0.8768 - val_loss: 0.6942 - val_acc: 0.7948


In [33]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.6941735058069229
Test accuracy: 0.7948
